In [1]:
from snowflake.snowpark.session import Session
import json
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session

d:\College\7374\FinalProject\env\lib\site-packages\snowflake\connector\options.py:107: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [ ]:
session

In [ ]:

# session.sql('create or replace stage dash_udfs').collect()
session.file.put("../temp/download.png",'@dash_models',overwrite=True,auto_compress=False)

[PutResult(source='download.png', target='download.png', source_size=4111, target_size=4112, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [3]:
# import pandas as pd
# df = pd.DataFrame({"FILE_NAME": ["image1"], "OUTPUT": ["cat"]})
# s = session.sql("session.sql('create database if not exists IMAGES')")
# s
# session.write_pandas(df, "IMAGES")

In [17]:
session.file.put('../temp/image_1.jpeg','@dash_models',overwrite=True,auto_compress=False)

[PutResult(source='image_1.jpeg', target='image_1.jpeg', source_size=26237, target_size=26240, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [18]:
import os
# Model Directory
directory = '../model/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        # Import trained model
        # session.file.put(f,'@dash_models',overwrite=True,auto_compress=False)
        session.add_import('@dash_models/'+filename)

In [38]:
session.file.put("../backend/images/image.jpg",'@dash_models',overwrite=True,auto_compress=False)


[PutResult(source='image.jpg', target='image.jpg', source_size=85949, target_size=85952, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [39]:
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit
import os
from snowflake.snowpark.types import  StringType, IntegerType

# session.add_import('@dash_models/m/tokenizer.json')
# session.add_import('@dash_models/pytorch_model.bin')
# session.add_import('@dash_files/mobilenetv3.py')
session.add_import('@dash_models/image.jpg')
@udf(name='image_caption_generator',session=session,replace=True,is_permanent=True,stage_location='@dash_models')
def image_caption_generator() -> str:
  from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
  import torch
  import sys
  from PIL import Image
  from io import BytesIO
  import requests
  import glob 
  IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
  import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
  # return os.listdir(import_dir)
  
  model_path = import_dir

  model = VisionEncoderDecoderModel.from_pretrained(model_path)
  feature_extractor = ViTFeatureExtractor.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  image_file_name = "/image.jpg"
  device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
  model.to(device)
  max_length = 16
  num_beams = 4
  gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
  images = []
  i_image = Image.open(model_path+image_file_name)
  if i_image.mode != "RGB":
    i_image = i_image.convert(mode="RGB")

  images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  # os.remove(model_path+"downloa")
  return preds[0]

In [ ]:
def generate_raw_caption(image_link : str) -> str:
    # Snowpark imports 
    from snowflake.snowpark.session import Session
    import json
    connection_parameters = json.load(open('connection.json'))
    session = Session.builder.configs(connection_parameters).create()
    # ... continued in backend

In [40]:
# image_links_in_str = ["https://as2.ftcdn.net/v2/jpg/03/03/62/45/1000_F_303624505_u0bFT1Rnoj8CMUSs8wMCwoKlnWlh5Jiq.jpg"]
predicted_label = session.sql('''SELECT image_caption_generator()''').collect()


In [41]:
predicted_label[0][0]

'a wooden table topped with a wooden table cloth'

In [4]:
session.add_packages(["transformers","Pillow"])

The version of package transformers in the local environment is 4.28.1, which does not fit the criteria for the requirement transformers. Your UDF might not work when the package version is different between the server and your local environment
The version of package pillow in the local environment is 9.5.0, which does not fit the criteria for the requirement pillow. Your UDF might not work when the package version is different between the server and your local environment


In [12]:
session.clear_imports()
session.clear_packages()